# مهمة تصنيف النصوص

في هذه الوحدة، سنبدأ بمهمة بسيطة لتصنيف النصوص باستخدام مجموعة بيانات **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: سنقوم بتصنيف عناوين الأخبار إلى واحدة من 4 فئات: العالم، الرياضة، الأعمال، والعلوم/التكنولوجيا.

## مجموعة البيانات

لتحميل مجموعة البيانات، سنستخدم واجهة برمجة التطبيقات **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

يمكننا الآن الوصول إلى أجزاء التدريب والاختبار من مجموعة البيانات باستخدام `dataset['train']` و `dataset['test']` على التوالي:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


دعونا نطبع أول 10 عناوين جديدة من مجموعة البيانات الخاصة بنا:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## تحويل النص إلى تمثيل عددي

الآن نحتاج إلى تحويل النص إلى **أرقام** يمكن تمثيلها كـ tensors. إذا أردنا تمثيل النص على مستوى الكلمات، يجب علينا القيام بخطوتين:

* استخدام **tokenizer** لتقسيم النص إلى **رموز**.
* بناء **قاموس** لهذه الرموز.

### تحديد حجم القاموس

في مثال مجموعة بيانات AG News، حجم القاموس كبير جدًا، حيث يتجاوز 100 ألف كلمة. بشكل عام، لا نحتاج إلى الكلمات التي نادرًا ما تظهر في النص — فقط عدد قليل من الجمل تحتوي عليها، ولن يتعلم النموذج منها. لذلك، من المنطقي تحديد حجم القاموس إلى عدد أصغر عن طريق تمرير وسيط إلى منشئ الـ vectorizer:

يمكن التعامل مع كلا الخطوتين باستخدام طبقة **TextVectorization**. دعونا ننشئ كائن الـ vectorizer، ثم نستدعي طريقة `adapt` لاستعراض جميع النصوص وبناء القاموس:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **ملاحظة** نحن نستخدم فقط جزءًا من مجموعة البيانات الكاملة لبناء المفردات. نقوم بذلك لتسريع وقت التنفيذ وعدم إبقائك منتظرًا. ومع ذلك، نحن نتحمل خطر أن بعض الكلمات من مجموعة البيانات الكاملة لن يتم تضمينها في المفردات، وسيتم تجاهلها أثناء التدريب. لذلك، استخدام حجم المفردات الكامل وتشغيل العملية على مجموعة البيانات بأكملها أثناء `adapt` يجب أن يزيد من الدقة النهائية، ولكن ليس بشكل كبير.

الآن يمكننا الوصول إلى المفردات الفعلية:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


باستخدام المُتجه، يمكننا بسهولة ترميز أي نص إلى مجموعة من الأرقام:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## تمثيل النص باستخدام حقيبة الكلمات

نظرًا لأن الكلمات تمثل المعاني، يمكننا أحيانًا فهم معنى نص معين بمجرد النظر إلى الكلمات الفردية فيه، بغض النظر عن ترتيبها في الجملة. على سبيل المثال، عند تصنيف الأخبار، فإن كلمات مثل *الطقس* و*الثلج* من المحتمل أن تشير إلى *توقعات الطقس*، بينما كلمات مثل *الأسهم* و*الدولار* قد تشير إلى *الأخبار المالية*.

**حقيبة الكلمات** (Bag-of-words أو BoW) هي أبسط طريقة لفهم تمثيل النصوص باستخدام المتجهات التقليدية. يتم ربط كل كلمة بمؤشر في المتجه، ويحتوي عنصر المتجه على عدد مرات ظهور كل كلمة في مستند معين.

![صورة توضح كيفية تمثيل حقيبة الكلمات باستخدام المتجهات في الذاكرة.](../../../../../translated_images/ar/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **ملاحظة**: يمكنك أيضًا التفكير في حقيبة الكلمات على أنها مجموع جميع المتجهات المشفرة بطريقة "واحد-ساخن" لكل كلمة فردية في النص.

فيما يلي مثال على كيفية إنشاء تمثيل حقيبة الكلمات باستخدام مكتبة Scikit Learn بلغة بايثون:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

يمكننا أيضًا استخدام مُوجّه Keras الذي قمنا بتعريفه أعلاه، لتحويل كل رقم كلمة إلى ترميز واحد فقط وجمع كل تلك المتجهات معًا:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **ملاحظة**: قد تتفاجأ بأن النتيجة تختلف عن المثال السابق. السبب هو أنه في مثال Keras، يتوافق طول المتجه مع حجم المفردات، التي تم إنشاؤها من مجموعة بيانات AG News بأكملها، بينما في مثال Scikit Learn قمنا ببناء المفردات من النص العينة مباشرة.


## تدريب المصنف BoW

الآن بعد أن تعلمنا كيفية بناء تمثيل حقيبة الكلمات (bag-of-words) لنصوصنا، دعونا ندرب مصنفًا يستخدم هذا التمثيل. أولاً، نحتاج إلى تحويل مجموعة البيانات الخاصة بنا إلى تمثيل حقيبة الكلمات. يمكن تحقيق ذلك باستخدام دالة `map` بالطريقة التالية:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

الآن دعونا نعرّف شبكة عصبية تصنيفية بسيطة تحتوي على طبقة خطية واحدة. حجم الإدخال هو `vocab_size`، وحجم الإخراج يتوافق مع عدد الفئات (4). نظرًا لأننا نقوم بحل مهمة تصنيف، فإن وظيفة التنشيط النهائية هي **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


نظرًا لأن لدينا 4 فئات، فإن دقة تتجاوز 80% تُعتبر نتيجة جيدة.

## تدريب مصنف كشبكة واحدة

بما أن الـ vectorizer هو أيضًا طبقة من طبقات Keras، يمكننا تعريف شبكة تتضمنه وتدريبها من البداية إلى النهاية. بهذه الطريقة، لسنا بحاجة إلى تحويل مجموعة البيانات باستخدام `map`، بل يمكننا ببساطة تمرير مجموعة البيانات الأصلية إلى مدخلات الشبكة.

> **ملاحظة**: سنظل بحاجة إلى تطبيق عمليات map على مجموعة البيانات لتحويل الحقول من القواميس (مثل `title`، `description` و`label`) إلى أزواج. ومع ذلك، عند تحميل البيانات من القرص، يمكننا إنشاء مجموعة بيانات بالهيكل المطلوب منذ البداية.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## ثنائيات الكلمات، ثلاثيات الكلمات و n-grams

أحد قيود نهج حقيبة الكلمات هو أن بعض الكلمات تكون جزءًا من تعبيرات متعددة الكلمات، على سبيل المثال، الكلمة "hot dog" لها معنى مختلف تمامًا عن الكلمات "hot" و "dog" في سياقات أخرى. إذا قمنا بتمثيل الكلمات "hot" و "dog" دائمًا باستخدام نفس المتجهات، فقد يؤدي ذلك إلى إرباك النموذج الخاص بنا.

لمعالجة هذا الأمر، يتم استخدام **تمثيلات n-gram** غالبًا في طرق تصنيف المستندات، حيث تكون تكرارات كل كلمة، أو ثنائية الكلمات، أو ثلاثية الكلمات ميزة مفيدة لتدريب المصنفات. في تمثيلات ثنائيات الكلمات، على سبيل المثال، سنضيف جميع أزواج الكلمات إلى المفردات، بالإضافة إلى الكلمات الأصلية.

فيما يلي مثال على كيفية إنشاء تمثيل حقيبة كلمات ثنائية باستخدام مكتبة Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

العيب الرئيسي في نهج n-gram هو أن حجم المفردات يبدأ في النمو بسرعة كبيرة جدًا. في الواقع، نحتاج إلى دمج تمثيل n-gram مع تقنية تقليل الأبعاد، مثل *التضمينات* (embeddings)، والتي سنتحدث عنها في الوحدة التالية.

لاستخدام تمثيل n-gram في مجموعة بيانات **AG News**، نحتاج إلى تمرير المعامل `ngrams` إلى منشئ `TextVectorization`. طول مفردات الـ bigram يكون **أكبر بكثير**، وفي حالتنا يتجاوز 1.3 مليون رمز! لذلك، من المنطقي أيضًا تحديد عدد رموز الـ bigram إلى رقم معقول.

يمكننا استخدام نفس الكود أعلاه لتدريب المصنف، ولكن ذلك سيكون غير فعال من حيث استهلاك الذاكرة. في الوحدة التالية، سنقوم بتدريب مصنف bigram باستخدام التضمينات. في هذه الأثناء، يمكنك تجربة تدريب مصنف bigram في هذا الدفتر (notebook) لترى ما إذا كان بإمكانك تحقيق دقة أعلى.


## حساب متجهات BoW تلقائيًا

في المثال أعلاه قمنا بحساب متجهات BoW يدويًا عن طريق جمع الترميزات ذات الواحد (one-hot encodings) للكلمات الفردية. ومع ذلك، فإن الإصدار الأحدث من TensorFlow يتيح لنا حساب متجهات BoW تلقائيًا عن طريق تمرير المعامل `output_mode='count` إلى منشئ الموجه. هذا يجعل تعريف وتدريب النموذج أسهل بشكل كبير:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## تكرار المصطلح - تكرار الوثيقة العكسي (TF-IDF)

في تمثيل BoW، يتم وزن ظهور الكلمات باستخدام نفس التقنية بغض النظر عن الكلمة نفسها. ومع ذلك، من الواضح أن الكلمات الشائعة مثل *a* و *in* أقل أهمية بكثير للتصنيف مقارنة بالمصطلحات المتخصصة. في معظم مهام معالجة اللغة الطبيعية، تكون بعض الكلمات أكثر أهمية من غيرها.

**TF-IDF** تعني **تكرار المصطلح - تكرار الوثيقة العكسي**. إنها نسخة معدلة من حقيبة الكلمات، حيث يتم استخدام قيمة عددية عائمة بدلاً من القيمة الثنائية 0/1 التي تشير إلى ظهور كلمة في وثيقة، وهذه القيمة ترتبط بتكرار ظهور الكلمة في المجموعة النصية.

بشكل أكثر رسمية، يتم تعريف الوزن $w_{ij}$ لكلمة $i$ في الوثيقة $j$ على النحو التالي:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
حيث:
* $tf_{ij}$ هو عدد مرات ظهور $i$ في $j$، أي قيمة BoW التي رأيناها سابقًا
* $N$ هو عدد الوثائق في المجموعة
* $df_i$ هو عدد الوثائق التي تحتوي على الكلمة $i$ في المجموعة بأكملها

تزداد قيمة TF-IDF $w_{ij}$ بشكل متناسب مع عدد مرات ظهور الكلمة في الوثيقة ويتم تعديلها بناءً على عدد الوثائق في المجموعة التي تحتوي على الكلمة، مما يساعد على التكيف مع حقيقة أن بعض الكلمات تظهر بشكل أكثر تكرارًا من غيرها. على سبيل المثال، إذا ظهرت الكلمة في *كل* الوثائق في المجموعة، فإن $df_i=N$، و $w_{ij}=0$، وسيتم تجاهل تلك المصطلحات تمامًا.

يمكنك بسهولة إنشاء تمثيل TF-IDF للنص باستخدام مكتبة Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

في كيراس، يمكن لطبقة `TextVectorization` حساب ترددات TF-IDF تلقائيًا عن طريق تمرير المعامل `output_mode='tf-idf'`. دعونا نكرر الكود الذي استخدمناه أعلاه لنرى ما إذا كان استخدام TF-IDF يزيد من الدقة:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## الخاتمة

على الرغم من أن تمثيلات TF-IDF توفر أوزانًا تعتمد على التكرار للكلمات المختلفة، إلا أنها غير قادرة على تمثيل المعنى أو الترتيب. كما قال اللغوي الشهير ج. ر. فيرث في عام 1935: "المعنى الكامل للكلمة دائمًا ما يكون سياقيًا، ولا يمكن أخذ أي دراسة للمعنى بعيدًا عن السياق على محمل الجد." سنتعلم لاحقًا في الدورة كيفية التقاط المعلومات السياقية من النص باستخدام نمذجة اللغة.



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
